In [ ]:
import pandas as pd
from datasets import Dataset
import numpy as np
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
import evaluate

In [ ]:
Data_Frame = pd.read_json('C:/Users/youss/Desktop/My Crap/Geek Mode/Projects/Coding Projects/My Personal Projects/AI Projects/AIC Arabic TSUM/Deliverables/Phase 1/test_set.jsonl', lines=True)
Data_Frame

In [ ]:
Data_Frame2 = Data_Frame
Data_Frame2

In [ ]:
Dataset_Paragraphs_Column_Name = 'paragraph'

In [ ]:
Model_Name = "facebook/mbart-large-50"
Tokenizer = AutoTokenizer.from_pretrained(Model_Name)
Model = AutoModelForSeq2SeqLM.from_pretrained(Model_Name)
Semantic_Model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
Model_Path = 'C:/Users/youss/Desktop/My Crap/Geek Mode/Projects/Coding Projects/My Personal Projects/AI Projects/AIC Arabic TSUM/Model/Generated Model/mBART_HYBRID_EASC_GPT_AICV1-2.pt'
Model.load_state_dict(torch.load(Model_Path))

In [ ]:
Paragraph_Max_Length = 1000
Summary_Max_Length = 300

In [ ]:
Rouge = evaluate.load("rouge")

In [ ]:
Model.device

In [ ]:
def Test_Model(Paragraph):
    Paragraph_text = Paragraph
    Paragraph_ids = Tokenizer(Paragraph_text, max_length=Paragraph_Max_Length, truncation=True, padding="longest", return_tensors="pt").to(Model.device)
    Summary_ids = Model.generate(Paragraph_ids['input_ids'],  num_beams=4,  decoder_start_token_id=Model.config.pad_token_id, max_length=300)

    Summary = Tokenizer.batch_decode(Summary_ids, skip_special_tokens=True)

    return Summary, Summary_ids

In [11]:
for index, row in Data_Frame.iterrows():
    paragraph = row['paragraph']

    generated_summary, summary_ids = Test_Model(paragraph) 

    Data_Frame2.at[index, 'summary'] = generated_summary[0]

    print(index, generated_summary[0])


KeyboardInterrupt: 

In [ ]:
data = Data_Frame2[['example_id', 'summary']]

data.to_json('C:/Users/youss/Desktop/My Crap/Geek Mode/Projects/Coding Projects/My Personal Projects/AI Projects/AIC Arabic TSUM/Deliverables/Phase 1/output/Hybrid-BBC-GPT-AIC/v2/predictions.jsonl', lines=True, orient='records', force_ascii=False)